<a href="https://colab.research.google.com/github/NHMSudara/Brachial-Plexus-Nerve-Trunks-Recognition/blob/master/KR260_Brachial_Plexus_Nerve_Trunks_Recognition_in_Ultrasound_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Prerequisites**

* Download Dataset: Download the dataset from the
provided link (https://drive.google.com/file/d/1gfuDLMw8FQdBo9QTaSOSr74daS5lxA2n/view?usp=drive_link) and Locate Dataset in your Google Drive.

* Connect the your Google Drive to  Google Colab and run following cells.



In [ ]:
!unzip /path/to/new_data.zip  # change the path

#**Image Load**

In [ ]:
from glob import glob
import os

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

dataset_path = "/content/new_data"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")
# Define the desired shape
target_shape_img = [96, 96, 3]
target_shape_mask = [96, 96, 1]

train_x, train_y = load_data(train_path)
valid_x, valid_y = load_data(valid_path)

#**Image Preprocess**

In [ ]:
import numpy as np
from PIL import Image
import cv2
from numpy import asarray
from sklearn.model_selection import train_test_split


def PreprocessData(img,mask,target_shape_img, target_shape_mask,path1):

    print(img)
    print(len(img))
    # Pull the relevant dimensions for image and mask
    m = len(img)                     # number of images
    i_h,i_w,i_c = target_shape_img   # pull height, width, and channels of image
    m_h,m_w,m_c = target_shape_mask  # pull height, width, and channels of mask

    # Define X and Y as number of images along with shape of one image
    X = np.zeros((m,i_h,i_w,i_c), dtype=np.float32)
    y = np.zeros((m,m_h,m_w,m_c), dtype=np.int32)
    index=0
    # Resize images and masks
    for file in img:
        # convert image into an array of desired shape (3 channels)
        sr = file.split('/')[5]
        dotsplit = sr.split('.')[0]
        sec=dotsplit.split('_')[-1]
        fir=dotsplit.split('_')[0]
        s=int(sec)
        f=int(fir)

        index= ((s*(f-1))+s)
        path = os.path.join(path1, file)
        single_img = Image.open(path).convert('RGB')
        single_img = single_img.resize((i_h,i_w))
        single_img = np.reshape(single_img,(i_h,i_w,i_c))
        single_img = single_img/255
        X[index] = single_img

        # convert mask into an array of desired shape (1 channel)

        single_mask_ind = mask[index]
        path = os.path.join(path1, single_mask_ind)
        single_mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        single_mask = cv2.resize(single_mask, dsize=(m_h, m_w), interpolation=cv2.INTER_NEAREST)
        single_mask = asarray(single_mask)
        single_mask = single_mask[..., tf.newaxis]
        single_mask = np.reshape(single_mask,(m_h,m_w,m_c))
        single_mask = (single_mask > 0)

        y[index] = single_mask
        index=index+1
    return X, y



X_train, Y_train = PreprocessData(train_x,train_y,target_shape_img, target_shape_mask, train_path)
X_test, Y_test = PreprocessData(valid_x,valid_y,target_shape_img, target_shape_mask, valid_path)


#**Loss Function**

In [ ]:

smooth = 1
def dice_coef(y_true, y_pred):
    y_true_float32 = tf.cast(y_true, tf.float32)
    y_pred_float32 = tf.cast(y_pred, tf.float32)
    y_true = tf.keras.layers.Flatten()( y_true_float32)
    y_pred = tf.keras.layers.Flatten()(y_pred_float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

#**CNN Model (Inc+ResU-Net )**

In [ ]:

from keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout,Activation, Concatenate,Add, Lambda
import joblib
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard

img_rows = 96
img_cols = 96
depth = 3

def block(prevlayer, a, b, pooling):
    conva = Conv2D(a, (5, 5), activation='relu', padding='same')(prevlayer)
    conva = BatchNormalization()(conva)
    conva = Conv2D(b, (5, 5), activation='relu', padding='same')(conva)
    conva = BatchNormalization()(conva)
    if True == pooling:
        conva = MaxPooling2D(pool_size=(2, 2))(conva)


    convb = Conv2D(a, (3, 3), activation='relu', padding='same')(prevlayer)
    convb = BatchNormalization()(convb)
    convb = Conv2D(b, (3, 3), activation='relu', padding='same')(convb)
    convb = BatchNormalization()(convb)
    if True == pooling:
        convb = MaxPooling2D(pool_size=(2, 2))(convb)

    convc = Conv2D(b, (1, 1), activation='relu', padding='same')(prevlayer)
    convc = BatchNormalization()(convc)
    if True == pooling:
        convc = MaxPooling2D(pool_size=(2, 2))(convc)

    convd = Conv2D(a, (2, 2), activation='relu', padding='same')(prevlayer)
    convd = BatchNormalization()(convd)
    convd = Conv2D(b, (1, 1), activation='relu', padding='same')(convd)
    convd = BatchNormalization()(convd)
    if True == pooling:
        convd = MaxPooling2D(pool_size=(2, 2))(convd)
    convs = Conv2D((b*4), (1, 1), activation='relu', padding='same')(prevlayer)
    if True == pooling:
        convs = MaxPooling2D(pool_size=(2, 2))(convs)
    up = concatenate([conva, convb, convc, convd])
    return up,convs


    return conv

def res_block(x1,x2):
  return Add()([x1,x2])


def create_inc_resunet_model():
    inputs = Input((img_rows, img_cols, depth))

    xx1,s = block(inputs, 16, 16, False)
    x1=res_block(xx1,s)


    xx2,s1 = block(xx1, 32, 32, True)
    x2=res_block(xx2,s1)


    xx3,s2 = block(xx2, 64, 64,True)
    x3=res_block(xx3,s2)


    xx4,s3 = block(xx3, 128, 128, True)
    x4=res_block(xx4,s3)


    up1 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4), x3], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up2 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), x2], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up3 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), x1], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv8)
    model = Model(inputs=[inputs], outputs=[conv10])


    return model

inc_resunet_model=create_inc_resunet_model()
print("Model created...")
inc_resunet_model.summary()



#**Train the CNN Model and save**

In [ ]:
inc_resunet_model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001,), loss=dice_loss, metrics=[dice_coef])
model_path = os.path.join("files", "inc_resmodel.h5")
csv_path = os.path.join("files", "inc_reslog.csv")
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]


file_name = 'inc_resUNet'
tensorboard = TensorBoard(log_dir="logs_{}".format(file_name))

history=inc_resunet_model.fit(X_train, Y_train, batch_size=32, epochs=20, verbose=1, shuffle=False,
              validation_data= [X_test, Y_test],callbacks=[callbacks,tensorboard])

Download trained model "inc_resmodel.h5"